In [1]:
import pickle
import numpy as np
import pandas as pd
from torch.utils.data import Dataset,DataLoader,RandomSampler,SequentialSampler,TensorDataset
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_util import *
from sklearn.metrics import roc_auc_score

In [7]:
batch_size = 16
num_train_epochs = 4
max_len = 64

In [3]:
with open('../Data/precessed_reviews.pickle', 'rb') as fp:
    precessed_reviews = pickle.load(fp)
with open('../Data/targets.pickle', 'rb') as fp:
    targets = pickle.load(fp) 
with open('../Data/precessed_reviews_test.pickle', 'rb') as fp:
    precessed_reviews_test = pickle.load(fp)     

In [4]:
class TextGenerator(Dataset):
    def __init__(self, reviews,targets,max_len=64):
        self.reviews = reviews
        self.targets = targets
        self.max_len = max_len
        self.length = [len(r)+2 for r in reviews]#+2 for CLS and SEP
        self.segment_ids = np.zeros(max_len,dtype=np.long)
        self.CLS = [101]
        self.SEP = [102]

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, index):
        review = self.reviews[index]
        len_ = self.length[index]
        if len_ < self.max_len:
            padding = [0] * (self.max_len - len_)
            input_mask = [1] * len_
            input_ids = self.CLS + review + self.SEP + padding
            input_mask += padding
        elif len_ == self.max_len:
            input_ids = self.CLS + review + self.SEP
            input_mask = [1] * len_
        else:
            start = np.random.randint(0,len_-self.max_len+1)
            input_ids = self.CLS + review[start:start+self.max_len-2] + self.SEP
            input_mask = [1] * self.max_len
        return np.array(input_ids,dtype=np.long),np.array(input_mask,dtype=np.long),\
                self.segment_ids,self.targets[index]

Dataset

In [5]:
train_size = 20000
train_gen = TextGenerator(precessed_reviews[:train_size],targets[:train_size],max_len)
train_gen = DataLoader(train_gen,batch_size,sampler=RandomSampler(train_gen),num_workers=2)

val_gen = TextGenerator(precessed_reviews[train_size:],targets[train_size:],max_len)
val_gen = DataLoader(val_gen,batch_size,sampler=SequentialSampler(train_gen),num_workers=2)

In [221]:
#inputs,inputs_mask,segment_ids,ys = next(iter(val_gen))

Model

In [6]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2).to('cuda:0')

Optimizer

In [8]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

In [9]:
num_train_optimization_steps = int(len(train_gen) * num_train_epochs)

In [10]:
opt = BertAdam(optimizer_grouped_parameters,
                     lr=5e-5,
                     warmup=0.1,
                     t_total=num_train_optimization_steps)

Loss

In [11]:
def loss_func(model,data):
    inputs,inputs_mask,segment_ids,ys = data
    loss = model(inputs,segment_ids,inputs_mask,ys)
    return loss

Training

In [12]:
model = fit(num_train_epochs, model, loss_func, opt, train_gen, val_gen)

epoch:0, train_loss:0.4492339190542698, val_loss:0.11312240362167358
epoch:1, train_loss:0.33975954596996305, val_loss:0.0856446847319603
epoch:2, train_loss:0.2796365533143282, val_loss:0.0888269692659378
epoch:3, train_loss:0.23037153880000114, val_loss:0.09252633154392242
epoch:4, train_loss:0.194124737149477, val_loss:0.08916087448596954
Training completed in 1054.6581809520721s


In [13]:
def batchedSeq(reviews,max_len):
    # used for TTA
    CLS = [101]
    SEP = [102]
    length = [len(i)+2 for i in reviews]
    input_ids = []
    input_masks = []
    seg_len = []
    for len_,review in zip(length,reviews):
        if len_ < max_len:
            padding = [0] * (max_len - len_)
            input_mask = [1] * len_
            input_id = CLS + review + SEP + padding
            input_mask += padding
            input_ids.append(input_id)
            input_masks.append(input_mask)
            seg_len.append(1)
        elif len_ == max_len:
            input_id = CLS + review + SEP
            input_mask = [1] * len_
            input_ids.append(input_id)
            input_masks.append(input_mask)
            seg_len.append(1)
        else:
            for j in range((len_-2)//(max_len-2)):
                input_id = CLS + review[j*(max_len-2):j*(max_len-2)+(max_len-2)] + SEP
                input_mask = [1] * max_len
                input_ids.append(input_id)
                input_masks.append(input_mask)
            seg_len.append((len_-2)//(max_len-2))
            
    return np.array(input_ids,dtype=np.long),np.array(input_masks,dtype=np.long),\
                np.zeros((len(input_ids),max_len),dtype=np.long),seg_len

In [14]:
inputs,inputs_mask,segment_ids,seg_len = batchedSeq(precessed_reviews[train_size:],max_len)
TTA_val_gen = TensorDataset(*[torch.tensor(i) for i in [inputs,segment_ids,inputs_mask]])
TTA_val_gen = DataLoader(TTA_val_gen,batch_size,num_workers=2)

In [16]:
def predict(model,dataloader,to_numpy=True):
    # dataloader return Xs only
    model.eval()
    with torch.no_grad():
        out = torch.cat([model(*data2cuda(data)) for data in dataloader])
        return out.cpu().detach().numpy() if to_numpy else out

In [17]:
yhat = predict(model,TTA_val_gen)

In [18]:
seg_len = np.cumsum(np.array(seg_len))
seg_len = np.insert(seg_len, 0, 0)

In [20]:
yhat_agg = []
for i in range(seg_len.shape[0]-1):
    yhat_agg.append(yhat[seg_len[i]:seg_len[i+1]].mean(0))
yhat_agg = np.array(yhat_agg)

In [21]:
# validation accuracy
np.sum(yhat_agg.argmax(1) == np.array(targets[train_size:]))/yhat_agg.shape[0]

0.9132

In [22]:
yhat_prob = np.exp(yhat)
yhat_prob = yhat_prob/yhat_prob.sum(1,keepdims=True)

In [23]:
yhat_agg2 = []
for i in range(seg_len.shape[0]-1):
    yhat_agg2.append(yhat_prob[seg_len[i]:seg_len[i+1]].mean(0))
yhat_agg2 = np.array(yhat_agg2)

In [24]:
# validation accuracy
np.sum(yhat_agg2.argmax(1) == np.array(targets[train_size:]))/yhat_agg.shape[0]

0.9096

In [25]:
# AUC
roc_auc_score(np.array(targets[train_size:]),yhat_agg2[:,1])

0.9638380238417107

Submission

In [26]:
submission = pd.read_csv('../Data/sampleSubmission.csv')

In [43]:
inputs,inputs_mask,segment_ids,seg_len = batchedSeq(precessed_reviews_test,max_len)
TTA_val_gen = TensorDataset(*[torch.tensor(i) for i in [inputs,segment_ids,inputs_mask]])
TTA_val_gen = DataLoader(TTA_val_gen,batch_size,num_workers=2)

In [46]:
ytest = predict(model,TTA_val_gen)

In [47]:
seg_len = np.cumsum(np.array(seg_len))
seg_len = np.insert(seg_len, 0, 0)

In [48]:
yhat_agg = []
for i in range(seg_len.shape[0]-1):
    yhat_agg.append(ytest[seg_len[i]:seg_len[i+1]].mean(0))
yhat_agg = np.array(yhat_agg)

In [52]:
yhat_prob = np.exp(yhat_agg)
yhat_prob = yhat_prob/yhat_prob.sum(1,keepdims=True)

In [53]:
submission.iloc[:,1] = yhat_prob[:,1]

In [54]:
submission.to_csv('../Submission/BERT.csv',index=False)